In [10]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd

import shutil
import sys
import os.path

import gurobipy

from pyomo.environ import *
from pyomo.gdp import *

In [11]:
TASKS = {
    ('Ταπ_1','Μπλε')   : {'dur': 45, 'prec': None},
    ('Ταπ_1','Κίτρινο') : {'dur': 10, 'prec': ('Ταπ_1','Μπλε')},
    ('Ταπ_2','Μπλε')   : {'dur': 20, 'prec': ('Ταπ_2','Πράσινο')},
    ('Ταπ_2','Πράσινο')  : {'dur': 10, 'prec': None},
    ('Ταπ_2','Κίτρινο') : {'dur': 34, 'prec': ('Ταπ_2','Μπλε')},
    ('Ταπ_3','Μπλε')   : {'dur': 12, 'prec': ('Ταπ_3','Κίτρινο')},
    ('Ταπ_3','Πράσινο')  : {'dur': 17, 'prec': ('Ταπ_3','Μπλε')},
    ('Ταπ_3','Κίτρινο') : {'dur': 28, 'prec': None},   
}

In [12]:
def jobshop_model(TASKS):
    
    model = ConcreteModel()

    model.TASKS = Set(initialize = TASKS.keys(), dimen=2)
    model.JOBS = Set(initialize = list(set([j for (j,m) in model.TASKS])))
    model.MACHINES = Set(initialize = list(set([m for (j,m) in model.TASKS])))
    
    model.TASKORDER = Set(initialize = model.TASKS * model.TASKS, dimen=4, 
        filter = lambda model, j, m, k, n: (k,n) == TASKS[(j,m)]['prec'])
    
    model.DISJUNCTIONS = Set(initialize = model.JOBS * model.JOBS * model.MACHINES, dimen=3,
        filter = lambda model, j, k, m: j < k and (j,m) in model.TASKS and (k,m) in model.TASKS)
    
    model.dur = Param(model.TASKS, initialize=lambda model, j, m: TASKS[(j,m)]['dur'])

    ub = sum([model.dur[j, m] for (j,m) in model.TASKS])
    
    # δημιουργώ μεταβλητές απόφασης
    model.makespan = Var(bounds=(0, ub))
    model.start = Var(model.TASKS, bounds=(0, ub))
    
    model.objective = Objective(expr = model.makespan, sense = minimize)

    model.finish = Constraint(model.TASKS, rule=lambda model, j, m:  
        model.start[j,m] + model.dur[j,m] <= model.makespan)
    
    model.preceding = Constraint(model.TASKORDER, rule=lambda model, j, m, k, n: 
        model.start[k,n] + model.dur[k,n] <= model.start[j,m])
    
    model.disjunctions = Disjunction(model.DISJUNCTIONS, rule=lambda model,j,k,m:
        [model.start[j,m] + model.dur[j,m] <= model.start[k,m], 
         model.start[k,m] + model.dur[k,m] <= model.start[j,m]])
    
    TransformationFactory('gdp.hull').apply_to(model)
    return model

jobshop_model(TASKS)

In [13]:
def jobshop_solve(model):
    SolverFactory('gurobi').solve(model)
    
    results = [{'Job': j,
                'Machine': m,
                'Start': model.start[j, m](), 
                'Duration': model.dur[j,m], 
                'Finish': model.start[(j, m)]() + model.dur[j,m]}
               for j,m in model.TASKS]
    return results

def jobshop(TASKS):
    return jobshop_solve(jobshop_model(TASKS))

results = jobshop(TASKS)
results

[{'Job': 'Ταπ_1',
  'Machine': 'Μπλε',
  'Start': 42.0,
  'Duration': 45,
  'Finish': 87.0},
 {'Job': 'Ταπ_1',
  'Machine': 'Κίτρινο',
  'Start': 87.0,
  'Duration': 10,
  'Finish': 97.0},
 {'Job': 'Ταπ_2',
  'Machine': 'Μπλε',
  'Start': 10.0,
  'Duration': 20,
  'Finish': 30.0},
 {'Job': 'Ταπ_2',
  'Machine': 'Πράσινο',
  'Start': 0.0,
  'Duration': 10,
  'Finish': 10.0},
 {'Job': 'Ταπ_2',
  'Machine': 'Κίτρινο',
  'Start': 30.0,
  'Duration': 34,
  'Finish': 64.0},
 {'Job': 'Ταπ_3',
  'Machine': 'Μπλε',
  'Start': 30.0,
  'Duration': 12,
  'Finish': 42.0},
 {'Job': 'Ταπ_3',
  'Machine': 'Πράσινο',
  'Start': 42.0,
  'Duration': 17,
  'Finish': 59.0},
 {'Job': 'Ταπ_3',
  'Machine': 'Κίτρινο',
  'Start': 0.0,
  'Duration': 28,
  'Finish': 28.0}]

In [14]:
schedule = pd.DataFrame(results)

print('\nSchedule by Job')
print(schedule.sort_values(by=['Job','Start']).set_index(['Job', 'Machine']))

print('\nSchedule by Machine')
print(schedule.sort_values(by=['Machine','Start']).set_index(['Machine', 'Job']))


Schedule by Job
               Start  Duration  Finish
Job   Machine                         
Ταπ_1 Μπλε      42.0        45    87.0
      Κίτρινο   87.0        10    97.0
Ταπ_2 Πράσινο    0.0        10    10.0
      Μπλε      10.0        20    30.0
      Κίτρινο   30.0        34    64.0
Ταπ_3 Κίτρινο    0.0        28    28.0
      Μπλε      30.0        12    42.0
      Πράσινο   42.0        17    59.0

Schedule by Machine
               Start  Duration  Finish
Machine Job                           
Κίτρινο Ταπ_3    0.0        28    28.0
        Ταπ_2   30.0        34    64.0
        Ταπ_1   87.0        10    97.0
Μπλε    Ταπ_2   10.0        20    30.0
        Ταπ_3   30.0        12    42.0
        Ταπ_1   42.0        45    87.0
Πράσινο Ταπ_2    0.0        10    10.0
        Ταπ_3   42.0        17    59.0
